### Observability

We use Arize Phoenix for observability. You can access tje Phoenix app in your browser by visiting `http://localhost:6006/` after running the following code.

In [25]:
# setup Phoenix
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Load the environment variables and the dataset

In [26]:
import os
from dotenv import load_dotenv

import pandas as pd

load_dotenv()

df = pd.read_csv("./data/prosperLoanData.csv")

In [27]:
df.shape

(113937, 81)

In [28]:
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


## Format dates

We want to format all columns with dates so that .dt can be used to access the date components.

In [29]:
df['ListingCreationDate'] = pd.to_datetime(df['ListingCreationDate'], format="mixed")
df['ClosedDate'] = pd.to_datetime(df['ClosedDate'], format="mixed")
df['DateCreditPulled'] = pd.to_datetime(df['DateCreditPulled'], format="mixed")
df['LoanOriginationDate'] = pd.to_datetime(df['LoanOriginationDate'], format="mixed")


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   ListingKey                           113937 non-null  object        
 1   ListingNumber                        113937 non-null  int64         
 2   ListingCreationDate                  113937 non-null  datetime64[ns]
 3   CreditGrade                          28953 non-null   object        
 4   Term                                 113937 non-null  int64         
 5   LoanStatus                           113937 non-null  object        
 6   ClosedDate                           55089 non-null   datetime64[ns]
 7   BorrowerAPR                          113912 non-null  float64       
 8   BorrowerRate                         113937 non-null  float64       
 9   LenderYield                          113937 non-null  float64       
 

In [31]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.settings import Settings
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [32]:
llm = TogetherLLM(
    model="meta-llama/Llama-3-70b-chat-hf", api_key=os.environ['TOGETHER_API_KEY']
)


Settings.llm = llm

We provide the model with the context of what each column represents.

In [33]:
df_info_str = """
ListingKey: Unique key for each listing, same value as the 'key' used in the listing object in the API.
ListingNumber: The number that uniquely identifies the listing to the public as displayed on the website.
ListingCreationDate: The date the listing was created.
CreditGrade: The Credit rating that was assigned at the time the listing went live. Applicable for listings pre-2009 period and will only be populated for those listings.
Term: The length of the loan expressed in months.
LoanStatus: The current status of the loan: Cancelled, Chargedoff, Completed, Current, Defaulted, FinalPaymentInProgress, PastDue. The PastDue status will be accompanied by a delinquency bucket.
ClosedDate: Closed date is applicable for Cancelled, Completed, Chargedoff and Defaulted loan statuses.
BorrowerAPR: The Borrower's Annual Percentage Rate (APR) for the loan.
BorrowerRate: The Borrower's interest rate for this loan.
LenderYield: The Lender yield on the loan. Lender yield is equal to the interest rate on the loan less the servicing fee.
EstimatedEffectiveYield: Effective yield is equal to the borrower interest rate (i) minus the servicing fee rate, (ii) minus estimated uncollected interest on charge-offs, (iii) plus estimated collected late fees. Applicable for loans originated after July 2009.
EstimatedLoss: Estimated loss is the estimated principal loss on charge-offs. Applicable for loans originated after July 2009.
EstimatedReturn: The estimated return assigned to the listing at the time it was created. Estimated return is the difference between the Estimated Effective Yield and the Estimated Loss Rate. Applicable for loans originated after July 2009.
ProsperRating (numeric): The Prosper Rating assigned at the time the listing was created: 0 - N/A, 1 - HR, 2 - E, 3 - D, 4 - C, 5 - B, 6 - A, 7 - AA. Applicable for loans originated after July 2009.
ProsperRating (Alpha): The Prosper Rating assigned at the time the listing was created between AA - HR. Applicable for loans originated after July 2009.
ProsperScore: A custom risk score built using historical Prosper data. The score ranges from 1-10, with 10 being the best, or lowest risk score. Applicable for loans originated after July 2009.
ListingCategory: The category of the listing that the borrower selected when posting their listing: 0 - Not Available, 1 - Debt Consolidation, 2 - Home Improvement, 3 - Business, 4 - Personal Loan, 5 - Student Use, 6 - Auto, 7- Other, 8 - Baby&Adoption, 9 - Boat, 10 - Cosmetic Procedure, 11 - Engagement Ring, 12 - Green Loans, 13 - Household Expenses, 14 - Large Purchases, 15 - Medical/Dental, 16 - Motorcycle, 17 - RV, 18 - Taxes, 19 - Vacation, 20 - Wedding Loans
BorrowerState: The two letter abbreviation of the state of the address of the borrower at the time the Listing was created.
Occupation: The Occupation selected by the Borrower at the time they created the listing.
EmploymentStatus: The employment status of the borrower at the time they posted the listing.
EmploymentStatusDuration: The length in months of the employment status at the time the listing was created.
IsBorrowerHomeowner: A Borrower will be classified as a homowner if they have a mortgage on their credit profile or provide documentation confirming they are a homeowner.
CurrentlyInGroup: Specifies whether or not the Borrower was in a group at the time the listing was created.
GroupKey: The Key of the group in which the Borrower is a member of. Value will be null if the borrower does not have a group affiliation.
DateCreditPulled: The date the credit profile was pulled.
CreditScoreRangeLower: The lower value representing the range of the borrower's credit score as provided by a consumer credit rating agency.
CreditScoreRangeUpper: The upper value representing the range of the borrower's credit score as provided by a consumer credit rating agency.
FirstRecordedCreditLine: The date the first credit line was opened.
CurrentCreditLines: Number of current credit lines at the time the credit profile was pulled.
OpenCreditLines: Number of open credit lines at the time the credit profile was pulled.
TotalCreditLinespast7years: Number of credit lines in the past seven years at the time the credit profile was pulled.
OpenRevolvingAccounts: Number of open revolving accounts at the time the credit profile was pulled.
OpenRevolvingMonthlyPayment: Monthly payment on revolving accounts at the time the credit profile was pulled.
InquiriesLast6Months: Number of inquiries in the past six months at the time the credit profile was pulled.
TotalInquiries: Total number of inquiries at the time the credit profile was pulled.
CurrentDelinquencies: Number of accounts delinquent at the time the credit profile was pulled.
AmountDelinquent: Dollars delinquent at the time the credit profile was pulled.
DelinquenciesLast7Years: Number of delinquencies in the past 7 years at the time the credit profile was pulled.
PublicRecordsLast10Years: Number of public records in the past 10 years at the time the credit profile was pulled.
PublicRecordsLast12Months: Number of public records in the past 12 months at the time the credit profile was pulled.
RevolvingCreditBalance: Dollars of revolving credit at the time the credit profile was pulled.
BankcardUtilization: The percentage of available revolving credit that is utilized at the time the credit profile was pulled.
AvailableBankcardCredit: The total available credit via bank card at the time the credit profile was pulled.
TotalTrades: Number of trade lines ever opened at the time the credit profile was pulled.
TradesNeverDelinquent: Number of trades that have never been delinquent at the time the credit profile was pulled.
TradesOpenedLast6Months: Number of trades opened in the last 6 months at the time the credit profile was pulled.
DebtToIncomeRatio: The debt to income ratio of the borrower at the time the credit profile was pulled. This value is Null if the debt to income ratio is not available. This value is capped at 10.01 (any debt to income ratio larger than 1000% will be returned as 1001%).
IncomeRange: The income range of the borrower at the time the listing was created.
IncomeVerifiable: The borrower indicated they have the required documentation to support their income.
StatedMonthlyIncome: The monthly income the borrower stated at the time the listing was created.
LoanKey: Unique key for each loan. This is the same key that is used in the API.
TotalProsperLoans: Number of Prosper loans the borrower at the time they created this listing. This value will be null if the borrower had no prior loans.
TotalProsperPaymentsBilled: Number of on time payments the borrower made on Prosper loans at the time they created this listing. This value will be null if the borrower had no prior loans.
OnTimeProsperPayments: Number of on time payments the borrower had made on Prosper loans at the time they created this listing. This value will be null if the borrower has no prior loans.
ProsperPaymentsLessThanOneMonthLate: Number of payments the borrower made on Prosper loans that were less than one month late at the time they created this listing. This value will be null if the borrower had no prior loans.
ProsperPaymentsOneMonthPlusLate: Number of payments the borrower made on Prosper loans that were greater than one month late at the time they created this listing. This value will be null if the borrower had no prior loans.
ProsperPrincipalBorrowed: Total principal borrowed on Prosper loans at the time the listing was created. This value will be null if the borrower had no prior loans.
ProsperPrincipalOutstanding: Principal outstanding on Prosper loans at the time the listing was created. This value will be null if the borrower had no prior loans.
ScorexChangeAtTimeOfListing: Borrower's credit score change at the time the credit profile was pulled. This will be the change relative to the borrower's last Prosper loan. This value will be null if the borrower had no prior loans.
LoanCurrentDaysDelinquent: The number of days delinquent.
LoanFirstDefaultedCycleNumber: The cycle the loan was charged off. If the loan has not charged off the value will be null.
LoanMonthsSinceOrigination: Number of months since the loan originated.
LoanNumber: Unique numeric value associated with the loan.
LoanOriginalAmount: The origination amount of the loan.
LoanOriginationDate: The date the loan was originated.
LoanOriginationQuarter: The quarter in which the loan was originated. Written in the format Q[quarter number] YYYY.
MemberKey: The unique key that is associated with the borrower. This is the same identifier that is used in the API member object.
MonthlyLoanPayment: The scheduled monthly loan payment.
LP_CustomerPayments: Pre charge-off cumulative gross payments made by the borrower on the loan. If the loan has charged off, this value will exclude any recoveries.
LP_CustomerPrincipalPayments: Pre charge-off cumulative principal payments made by the borrower on the loan. If the loan has charged off, this value will exclude any recoveries.
LP_InterestandFees: Pre charge-off cumulative interest and fees paid by the borrower. If the loan has charged off, this value will exclude any recoveries.
LP_ServiceFees: Cumulative service fees paid by the investors who have invested in the loan.
LP_CollectionFees: Cumulative collection fees paid by the investors who have invested in the loan.
LP_GrossPrincipalLoss: The gross charged off amount of the loan.
LP_NetPrincipalLoss: The principal that remains uncollected after any recoveries.
LP_NonPrincipalRecoverypayments: The interest and fee component of any recovery payments. The current payment policy applies payments in the following order: Fees, interest, principal.
PercentFunded: Percent the listing was funded.
Recommendations: Number of recommendations the borrower had at the time the listing was created.
InvestmentFromFriendsCount: Number of friends that made an investment in the loan.
InvestmentFromFriendsAmount: Dollar amount of investments that were made by friends.
Investors: The number of investors that funded the loan.
"""

In [34]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. GIVE ONLY THE EXPRESSION AND NOTHING ELSE AS AN ANSWER.\n"
    "5. Do not quote the expression.\n"
    "6. The data is stored in the `df` variable"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "This is the definition of all of the columns in the dataframe, there are no other columns besides this:\n"
    "{df_info_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

In [35]:
from llama_index.core.prompts import PromptTemplate

In [36]:
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5), df_info_str=df_info_str
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

### Custom Query Pipeline

We will use a custom query pipeline to preprocess the data before it is passed to the model. This will allow us to initially generate a query, run the query, and then synthesize a response based on the query result

In [37]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": Settings.llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": Settings.llm,
    },
    verbose=True,
)

qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])

qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)

qp.add_link("response_synthesis_prompt", "llm2")

### Querying

After running all of the following queries, you can view the traces in the Phoenix app on your browser to see exactly what is happening at each step.

In [14]:
response = qp.run(query_str="What are the top earning occupations per month?")
print(response)

> Running module input with input: 
query_str: What are the top earning occupations per month?

> Running module pandas_prompt with input: 
query_str: What are the top earning occupations per month?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df.groupby('Occupation')['StatedMonthlyIncome'].mean().sort_values(ascending=False)

> Running module response_synthesis_prompt with input: 
query_str: What are the top earning occupations per month?
pandas_instructions: assistant: df.groupby('Occupation')['StatedMonthlyIncome'].mean().sort_values(ascending=False)
pandas_output: Occupation
Doctor                         14936.942814
Attorney                       11570.032027
Judge                          10808.136364
Executi

In [38]:
response = qp.run(query_str="What is the average loan amount?")
print(response)

> Running module input with input: 
query_str: What is the average loan amount?

> Running module pandas_prompt with input: 
query_str: What is the average loan amount?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df.LoanOriginalAmount.mean()

> Running module response_synthesis_prompt with input: 
query_str: What is the average loan amount?
pandas_instructions: assistant: df.LoanOriginalAmount.mean()
pandas_output: 8337.013849759078

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: What is the average loan amount?

Pandas Instructions (optional):
df.LoanOriginalAmount.mean()

Pandas Output: 8337.013849...

assistant: The average loan amount i

In [40]:
response = qp.run(query_str="What is the average loan amount for borrowers that are self-employed?")
print(response)


> Running module input with input: 
query_str: What is the average loan amount for borrowers that are self-employed?

> Running module pandas_prompt with input: 
query_str: What is the average loan amount for borrowers that are self-employed?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df.loc[df['EmploymentStatus'] == 'Self-employed', 'LoanOriginalAmount'].mean()

> Running module response_synthesis_prompt with input: 
query_str: What is the average loan amount for borrowers that are self-employed?
pandas_instructions: assistant: df.loc[df['EmploymentStatus'] == 'Self-employed', 'LoanOriginalAmount'].mean()
pandas_output: 8122.836485164656

> Running module llm2 with input: 
messages: Given an input question, synth

In [18]:
response = qp.run(query_str="Summarize the maximum, minimum, and average BorrowerAPR for loans")
print(response)

> Running module input with input: 
query_str: Summarize the maximum, minimum, and average BorrowerAPR for loans

> Running module pandas_prompt with input: 
query_str: Summarize the maximum, minimum, and average BorrowerAPR for loans

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df['BorrowerAPR'].describe()

> Running module response_synthesis_prompt with input: 
query_str: Summarize the maximum, minimum, and average BorrowerAPR for loans
pandas_instructions: assistant: df['BorrowerAPR'].describe()
pandas_output: count    113912.000000
mean          0.218828
std           0.080364
min           0.006530
25%           0.156290
50%           0.209760
75%           0.283810
max           0.512290
Name: BorrowerAP...

>

In [19]:
response = qp.run(query_str="Is there a correlation between loan original amount and debt to income ratio?")
print(response)

> Running module input with input: 
query_str: Is there a correlation between loan original amount and debt to income ratio?

> Running module pandas_prompt with input: 
query_str: Is there a correlation between loan original amount and debt to income ratio?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df['LoanOriginalAmount'].corr(df['DebtToIncomeRatio'])

> Running module response_synthesis_prompt with input: 
query_str: Is there a correlation between loan original amount and debt to income ratio?
pandas_instructions: assistant: df['LoanOriginalAmount'].corr(df['DebtToIncomeRatio'])
pandas_output: 0.01011222449159212

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from 

In [20]:
response = qp.run(query_str="Which year recorded the highest number of loan requests?")
print(response)

> Running module input with input: 
query_str: Which year recorded the highest number of loan requests?

> Running module pandas_prompt with input: 
query_str: Which year recorded the highest number of loan requests?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: df.LoanOriginationDate.dt.year.value_counts().idxmax()

> Running module response_synthesis_prompt with input: 
query_str: Which year recorded the highest number of loan requests?
pandas_instructions: assistant: df.LoanOriginationDate.dt.year.value_counts().idxmax()
pandas_output: 2013

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: Which year recorded the highest number of loan reque

In [43]:
response = qp.run(query_str="How many listings of loans in Q3 2013?")
print(response)

> Running module input with input: 
query_str: How many listings of loans in Q3 2013?

> Running module pandas_prompt with input: 
query_str: How many listings of loans in Q3 2013?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
                ListingKey  ListingNumber     ListingCreationDate Credit...

> Running module pandas_output_parser with input: 
input: assistant: (len(df[(df.LoanOriginationQuarter == 3) & (df.LoanOriginationDate.dt.year == 2013)]))

> Running module response_synthesis_prompt with input: 
query_str: How many listings of loans in Q3 2013?
pandas_instructions: assistant: (len(df[(df.LoanOriginationQuarter == 3) & (df.LoanOriginationDate.dt.year == 2013)]))
pandas_output: 0

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: How many listings of loans in Q3 2013?

Panda